In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy 
from cartopy.util import add_cyclic_point
import cartopy.mpl.ticker as cticker
import xesmf as xe
import matplotlib.colors as colors
from scipy.stats import pearsonr

In [2]:
def regrid(ds, var, lat_step=0.1, lon_step=0.1):
    # Create new lons and lats on a regular grid
    new_lat=np.arange(-90,90.1,lat_step)
    new_lon=np.arange(0,360.1,lon_step)

    # Create an xarray.Dataset that defines the new grid
    ds_out = xr.Dataset({'lat': (['lat'], new_lat),
                    'lon': (['lon'], new_lon)})
    
    # Regrid the data to the new grid 
    regridder = xe.Regridder(ds, ds_out, 'bilinear', reuse_weights=True)
    dr = regridder(ds).to_dataset(name=var)
    
    dr = dr.sel(lat=slice(0,90))

    return dr

In [25]:
file='/home/ubuntu/asr15km.anl.2D.anoms.nc'
anoms=xr.open_dataset(file)

In [39]:
albedos = anoms['ALBEDO']

In [51]:
nt = len(albedos['time'])

In [60]:
albedosr = []
for t in range(0,nt):
    albedosr.append(regrid(albedos[t], 'ALBEDO'))

In [61]:
ds_ars = xr.concat(albedosr, dim='time')
ds_ars.to_netcdf('/home/ubuntu/asr15km.anl.2D.latlon.anoms.ALBEDO.nc')